# 日経平均株価 先物 15分足

In [1]:
import sys

path_up = ".."
if not path_up in sys.path:
    sys.path.append(path_up)

In [2]:
import datetime
import os

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import mplfinance as mpf
import numpy as np
import pandas as pd
import pingouin as pg
import seaborn as sns
import yfinance as yf

from funcs.technical import calc_robust_bollinger

In [3]:
code = "NIY=F"
#code = "NIYH26.CME"
symbol = "%s" % code
ticker1 = yf.Ticker(symbol)
df = ticker1.history(period="2d", interval="15m")
df.index = df.index.tz_convert("Asia/Tokyo")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2026-02-19 14:00:00+09:00,57580.0,57615.0,57560.0,57600.0,137,0.0,0.0
2026-02-19 14:15:00+09:00,57610.0,57625.0,57580.0,57620.0,53,0.0,0.0
2026-02-19 14:30:00+09:00,57620.0,57640.0,57545.0,57565.0,68,0.0,0.0
2026-02-19 14:45:00+09:00,57560.0,57590.0,57550.0,57555.0,45,0.0,0.0
2026-02-19 15:00:00+09:00,57560.0,57560.0,57480.0,57480.0,85,0.0,0.0
...,...,...,...,...,...,...,...
2026-02-21 04:45:00+09:00,57110.0,57145.0,57090.0,57095.0,171,0.0,0.0
2026-02-21 05:00:00+09:00,57090.0,57165.0,57090.0,57150.0,263,0.0,0.0
2026-02-21 05:15:00+09:00,57140.0,57145.0,57090.0,57105.0,164,0.0,0.0


In [4]:
dt1 = df.head(1).index[0]
dt2 = dt1 + datetime.timedelta(days=1)
dt_start = pd.to_datetime(
    f"{dt2.year:04d}-{dt2.month:02d}-{dt2.day:02d} 15:30:00+09:00"
)
df = df[df.index >= dt_start]
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2026-02-20 15:30:00+09:00,56850.0,56860.0,56790.0,56835.0,252,0.0,0.0
2026-02-20 15:45:00+09:00,56835.0,56885.0,56800.0,56855.0,169,0.0,0.0
2026-02-20 16:00:00+09:00,56850.0,56880.0,56830.0,56870.0,211,0.0,0.0
2026-02-20 16:15:00+09:00,56860.0,56860.0,56835.0,56835.0,62,0.0,0.0
2026-02-20 16:30:00+09:00,56820.0,56840.0,56805.0,56830.0,87,0.0,0.0
2026-02-20 16:45:00+09:00,56835.0,56835.0,56800.0,56825.0,146,0.0,0.0
2026-02-20 17:00:00+09:00,56825.0,56900.0,56810.0,56815.0,356,0.0,0.0
2026-02-20 17:15:00+09:00,56825.0,56905.0,56765.0,56895.0,176,0.0,0.0
2026-02-20 17:30:00+09:00,56900.0,56940.0,56860.0,56940.0,260,0.0,0.0


In [5]:
dt = df.tail(1).index[0]
year_str = f"{dt.year:04d}"
date_str = f"{dt.month:02d}{dt.day:02d}"
date_str

'0221'

## ローソク足チャート（小さいサイズ）

In [6]:
FONT_PATH = "../fonts/RictyDiminished-Regular.ttf"
fm.fontManager.addfont(FONT_PATH)

# FontPropertiesオブジェクト生成（名前の取得のため）
font_prop = fm.FontProperties(fname=FONT_PATH)
font_prop.get_name()

plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["font.size"] = 10

fig, ax = plt.subplots(figsize=(6, 4))
mpf.plot(
    df,
    type="candle",
    style="default",
    datetime_format="%H:%M",
    xrotation=0,
    update_width_config=dict(candle_linewidth=0.75),
    ax=ax,
)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
ax.set_xlabel(f"Time from {dt_start}")
ax.set_ylabel("JPY")
ax.grid()

if "shortName" in ticker1.info:
    ax.set_title(f"{ticker1.info['shortName']} ({symbol})")
elif "longName" in ticker1.info:
    ax.set_title(f"{ticker1.info['longName']} ({symbol})")
else:
    ax.set_title(f"{symbol}")

plt.tight_layout()
plt.savefig(os.path.join(f"{year_str}", f"{date_str}_report_15min_chart_{code}.png"))
plt.show()